In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('./exercises/earthquakes.csv')

In [18]:
filtered_df = df.query("parsed_place == 'Japan'and mag >= 4.9and magType == 'mb'")

print(filtered_df)

      mag magType           time                         place  tsunami  \
1563  4.9      mb  1538977532250  293km ESE of Iwo Jima, Japan        0   
2576  5.4      mb  1538697528010    37km E of Tomakomai, Japan        0   
3072  4.9      mb  1538579732490     15km ENE of Hasaki, Japan        0   
3632  4.9      mb  1538450871260    53km ESE of Hitachi, Japan        0   

     parsed_place  
1563        Japan  
2576        Japan  
3072        Japan  
3632        Japan  


In [27]:
count_by_bin = (df.query("magType == 'ml'")
                .assign(mag_bin=pd.cut(df['mag'], bins=range(0, 11), right=True))
                .groupby('mag_bin')
                .size())
print(count_by_bin)

mag_bin
(0, 1]     2207
(1, 2]     3105
(2, 3]      862
(3, 4]      122
(4, 5]        2
(5, 6]        1
(6, 7]        0
(7, 8]        0
(8, 9]        0
(9, 10]       0
dtype: int64


In [32]:
faang = pd.read_csv('./exercises/faang.csv')
faang['date'] = pd.to_datetime(faang['date'])
faang.dtypes

ticker            object
date      datetime64[ns]
high             float64
low              float64
open             float64
close            float64
volume           float64
dtype: object

In [37]:
faang.set_index('date', inplace=True)
monthly_data = (faang.groupby('ticker').resample('M').agg({
                    'open': np.mean,
                    'high': np.max,
                    'low': np.min,
                    'close': np.mean,
                    'volume': np.sum
                }))

print(monthly_data)

                          open         high          low        close  \
ticker date                                                             
AAPL   2018-01-31    43.505357    45.025002    41.174999    43.501309   
       2018-02-28    41.819079    45.154999    37.560001    41.909737   
       2018-03-31    43.761786    45.875000    41.235001    43.624048   
       2018-04-30    42.441310    44.735001    40.157501    42.458572   
       2018-05-31    46.239091    47.592499    41.317501    46.384205   
       2018-06-30    47.180119    48.549999    45.182499    47.155357   
       2018-07-31    47.549048    48.990002    45.855000    47.577857   
       2018-08-31    53.121739    57.217499    49.327499    53.336522   
       2018-09-30    55.582763    57.417500    53.825001    55.518421   
       2018-10-31    55.300000    58.367500    51.522499    55.211413   
       2018-11-30    47.954881    55.590000    42.564999    47.808929   
       2018-12-31    41.310789    46.235001    36.6

In [40]:
earthquakes = pd.read_csv('./exercises/earthquakes.csv')

crosstab_result = pd.crosstab(
    index=earthquakes['tsunami'],
    columns=earthquakes['magType'],
    values=earthquakes['mag'],
    aggfunc=np.max 
)

print(crosstab_result)

magType   mb  mb_lg    md   mh   ml  ms_20    mw  mwb  mwr  mww
tsunami                                                        
0        5.6    3.5  4.11  1.1  4.2    NaN  3.83  5.8  4.8  6.0
1        6.1    NaN   NaN  NaN  5.1    5.7  4.41  NaN  NaN  7.5


In [43]:
rolling60 = faang.groupby('ticker').apply(
    lambda x: x.rolling(window='60D').agg({
        'open': np.mean,
        'high': np.max,
        'low': np.min,
        'close': np.mean
    })
)

print(rolling60)

                         open        high         low       close
ticker date                                                      
AAPL   2018-01-02   42.540001   43.075001   42.314999   43.064999
       2018-01-03   42.836250   43.637501   42.314999   43.061249
       2018-01-04   42.935833   43.637501   42.314999   43.126666
       2018-01-05   43.041875   43.842499   42.314999   43.282499
       2018-01-08   43.151000   43.902500   42.314999   43.343500
...                       ...         ...         ...         ...
NFLX   2018-12-24  283.509251  332.049988  233.679993  281.931750
       2018-12-26  281.844501  332.049988  231.229996  280.777750
       2018-12-27  281.070489  332.049988  231.229996  280.162927
       2018-12-28  279.916342  332.049988  231.229996  279.461464
       2018-12-31  278.430770  332.049988  231.229996  277.451539

[1255 rows x 4 columns]


In [46]:
pivot_table = faang.pivot_table(index='ticker')

print(pivot_table)

              close         high          low         open        volume
ticker                                                                  
AAPL      47.263357    47.748526    46.795877    47.277859  1.360803e+08
AMZN    1641.726176  1662.839839  1619.840519  1644.072709  5.648994e+06
FB       171.510956   173.613347   169.303148   171.472948  2.765860e+07
GOOG    1113.225134  1125.777606  1101.001658  1113.554101  1.741965e+06
NFLX     319.290319   325.219322   313.187330   319.620558  1.146962e+07


In [51]:
amzn = faang.loc['2018-Q4'].query("ticker == 'AMZN'")

z_amzn = amzn.drop(columns='ticker').apply(
    lambda x: (x - x.mean()) / x.std()
)

print(z_amzn)

                high       low      open     close    volume
date                                                        
2018-10-01  2.368006  2.502113  2.337813  2.385848 -1.630411
2018-10-02  2.227302  2.247433  2.190795  2.155037 -0.861879
2018-10-03  2.058955  2.139987  2.068570  2.025489 -0.920345
2018-10-04  1.819474  1.781561  1.850048  1.722816 -0.126582
2018-10-05  1.628173  1.554416  1.642819  1.584748 -0.298771
...              ...       ...       ...       ...       ...
2018-12-24 -2.159820 -2.187566 -2.179582 -2.226185 -0.141238
2018-12-26 -1.611714 -1.810493 -2.026617 -1.339674  1.123063
2018-12-27 -1.641276 -1.626703 -1.456521 -1.404343  0.849827
2018-12-28 -1.325261 -1.231588 -1.328549 -1.289951  0.496102
2018-12-31 -1.273456 -0.975763 -1.078283 -1.122691 -0.246405

[63 rows x 5 columns]


In [65]:
faang = pd.read_csv('./exercises/faang.csv')
faang['date'] = pd.to_datetime(faang['date'])
event_data = pd.DataFrame({
    'ticker': 'FB',
    'date': ['2018-07-25', '2018-03-19', '2018-03-20'],
    'event': [
        'Disappointing user growth announced after close.',
        'Cambridge Analytica story',
        'FTC investigation'
    ]
})

In [66]:
merged_data = pd.merge(
    faang.set_index(['date', 'ticker']), 
    event_data.set_index(['date', 'ticker']), 
    left_index=True, 
    right_index=True, 
    how='outer', 
    sort=False
).reset_index()

print(merged_data)

           date ticker         high          low         open        close  \
0    2018-01-02   AAPL    43.075001    42.314999    42.540001    43.064999   
1    2018-01-02   AMZN  1190.000000  1170.510010  1172.000000  1189.010010   
2    2018-01-02     FB   181.580002   177.550003   177.679993   181.419998   
3    2018-01-02   NFLX   201.649994   195.419998   196.100006   201.070007   
4    2018-01-03   AAPL    43.637501    42.990002    43.132500    43.057499   
...         ...    ...          ...          ...          ...          ...   
1253 2018-12-28   GOOG  1055.560059  1033.099976  1049.619995  1037.079956   
1254 2018-12-31   GOOG  1052.699951  1023.590027  1050.959961  1035.609985   
1255 2018-07-25     FB   218.619995   214.270004   215.720001   217.500000   
1256 2018-03-19     FB   177.169998   170.059998   177.009995   172.559998   
1257 2018-03-20     FB   170.199997   161.949997   167.470001   168.149994   

           volume event  
0     102223600.0   NaN  
1       269

C:\Users\James\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3587: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  uniq_tuples = lib.fast_unique_multiple([self._values, other._values], sort=sort)


In [68]:
print(event_data)

  ticker        date                                             event
0     FB  2018-07-25  Disappointing user growth announced after close.
1     FB  2018-03-19                         Cambridge Analytica story
2     FB  2018-03-20                                 FTC investigation


In [82]:
faang = pd.read_csv('./exercises/faang.csv')
faang['date'] = pd.to_datetime(faang['date'])
faang.sort_values(['ticker', 'date'], inplace=True)
transform_faang = faang.groupby('ticker').transform(lambda x: x / x.iloc[0])
print(transform_faang)

          high       low      open     close    volume
251   1.000000  1.000000  1.000000  1.000000  1.000000
252   1.013059  1.015952  1.013928  0.999826  1.155033
253   1.006790  1.016661  1.013987  1.004470  0.877864
254   1.017818  1.022392  1.019276  1.015906  0.925814
255   1.019211  1.027591  1.024624  1.012133  0.804816
...        ...       ...       ...       ...       ...
999   1.242995  1.195783  1.234064  1.163177  0.870583
1000  1.262088  1.183246  1.192861  1.261600  1.313288
1001  1.267493  1.228636  1.275421  1.271050  1.115648
1002  1.298835  1.278272  1.315349  1.273586  1.002362
1003  1.339450  1.330468  1.326670  1.331178  1.231788

[1255 rows x 5 columns]


In [83]:
covid = pd.read_csv('./exercises/covid19_cases.csv')
covid['date'] = pd.to_datetime(covid['dateRep'])
covid.set_index('date', inplace=True)
covid.replace({'United_States_of_America': 'USA', 'United_Kingdom': 'UK'}, inplace=True)
covid.sort_index(inplace=True)

In [109]:
import pandas as pd

covid = pd.read_csv('./exercises/covid19_cases.csv')
covid['date'] = pd.to_datetime(covid['dateRep'])
covid.set_index('date', inplace=True)
covid.replace({'countriesAndTerritories': {'United_States_of_America': 'USA', 'United_Kingdom': 'UK'}}, inplace=True)
covid.sort_index(inplace=True)

In [98]:
# Sum the cases by date and country
grouped = covid.groupby('countriesAndTerritories').cases.sum().nlargest(5).index

covid[covid.countriesAndTerritories.isin(grouped)].groupby('countriesAndTerritories').cases.idxmax()

countriesAndTerritories
Brazil   2020-07-30
India    2020-09-17
Peru     2020-08-17
Russia   2020-07-18
USA      2020-07-25
Name: cases, dtype: datetime64[ns]

In [120]:
# Calculate 7-day rolling mean of the daily case difference
seven_day_avg = covid[covid['countriesAndTerritories'].isin(grouped)] \
    .groupby(['countriesAndTerritories', 'date']).cases.sum() \
    .groupby('countriesAndTerritories').apply(lambda x: x.diff().rolling(7).mean())

# Get the last week's data
last_week_seven_day_avg = seven_day_avg.groupby('countriesAndTerritories').last('1W')

print(last_week_seven_day_avg)


countriesAndTerritories
Brazil     6245.428571
India     13938.571429
Peru        -98.285714
Russia      786.285714
USA        6865.857143
Name: cases, dtype: float64


In [122]:
remove_china = covid[covid['countriesAndTerritories'] != 'China']
remove_china.groupby('countriesAndTerritories')['cases'].idxmax()

countriesAndTerritories
Afghanistan      2020-05-16
Albania          2020-08-25
Algeria          2020-07-25
Andorra          2020-09-15
Angola           2020-06-08
                    ...    
Vietnam          2020-07-31
Western_Sahara   2020-07-14
Yemen            2020-06-16
Zambia           2020-07-18
Zimbabwe         2020-02-08
Name: cases, Length: 209, dtype: datetime64[ns]

In [125]:
cumulative_cases = covid.groupby('countriesAndTerritories')['cases'].sum()
cumulative_cases.rank(pct=True).sort_values(ascending=False)


countriesAndTerritories
USA                            1.000000
India                          0.995238
Brazil                         0.990476
Russia                         0.985714
Peru                           0.980952
                                 ...   
Greenland                      0.023810
Montserrat                     0.016667
Falkland_Islands_(Malvinas)    0.016667
Holy_See                       0.009524
Anguilla                       0.004762
Name: cases, Length: 210, dtype: float64